In [ ]:
import tkinter as tk # Biblioteca gráfica para la creación de interfaces gráficas
import serial # Permite la comunicación con el puerto serial de Arduino
import time # Para gestionar pausas

In [ ]:
# Configuración de la comunicación con el puerto serial
arduino = serial.Serial("COM3", 9600)
time.sleep(2)   # Espera a que la conexión con Arduino se establezca

# Función para encender el flujo de agua
def encender():
    arduino.write(b'a')

# Función para apagar el flujo de agua    
def apagar():
    arduino.write(b'q')
   
# Creación de la ventana principal
window = tk.Tk()
window.title("Control de la bomba de agua") # Poner un Título a la ventana
window.geometry("300x150") # Dimensiones de la ventana
window.configure(bg="lightgray") # Color de fondo

# Creación de botones para controlar la bomba
bttnEncender = tk.Button(window, text="Encender", command=encender, bg="white", fg="black", font=("Arial", 12, "bold"), bd=3)
bttnEncender.place(x=50, y=75)

bttnApagar = tk.Button(window, text="Apagar", command=apagar, bg="white", fg="black", font=("Arial", 12, "bold"), bd=3)
bttnApagar.place(x=200, y=75)

# Inicio del bucle principal de la interfaz gráfica
window.mainloop()